In [5]:
# The C2QA pacakge is currently not published to PyPI. 
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Implement Jaynes-Cummings model simulation

In [6]:
import c2qa
import numpy
import qiskit
import scipy.sparse.linalg
from scipy.sparse import kron

num_qumodes = 1
num_qubits_per_qumode = 4
num_qubits = 2
total_qubits = num_qumodes * num_qubits_per_qumode + num_qubits

## Define functions to calculate Hamiltonian and simulate circuit in loop

In [7]:
def simulate_jaynes_cummings(c1, c2, d1, d2, t, psi = None):
    qmr = c2qa.QumodeRegister(num_qumodes=num_qumodes, num_qubits_per_qumode=num_qubits_per_qumode)
    qr = qiskit.QuantumRegister(size=num_qubits)
    cr = qiskit.ClassicalRegister(size=1)
    circuit = c2qa.CVCircuit(qmr, qr, cr)

    if psi:
        circuit.initialize(psi)

    circuit.rz(-1 * c1 * t, qr[0])
    circuit.rz(-1 * c2 * t, qr[1])
    circuit.cx(qr[0], qr[1])
    circuit.h(qr[0])
    circuit.cv_cnd_d(d1, d1, qr[0], qmr[0], inverse=True)
    circuit.cx(qr[1], qr[0])
    circuit.cv_cnd_d(d1, d1, qr[0], qmr[0])
    circuit.cx(qr[1], qr[0])
    circuit.cv_cnd_d(d2, d2, qr[0], qmr[0], inverse=True)
    circuit.cx(qr[1], qr[0])
    circuit.cv_cnd_d(d2, d2, qr[0], qmr[0])
    circuit.cx(qr[1], qr[0])
    circuit.h(qr[0])
    circuit.cx(qr[0], qr[1])

    state, result = c2qa.util.simulate(circuit)
    return circuit, state, result

In [8]:
def calculate_hamiltonian(c1, c2, d, circuit: c2qa.CVCircuit):
    X = scipy.sparse.csr_matrix([[0, 1], [1, 0]])      # Pauli X
    Y = scipy.sparse.csr_matrix([[0, -1j], [-1j, 0]])  # Pauli Y
    Z = scipy.sparse.csr_matrix([[1, 0], [0, -1]])     # Pauli Z

    b = circuit.ops.a
    b_dag = circuit.ops.a_dag

    # Hamiltonian
    #   H = (c1 * a_dag1 * a1) + (c2 * a_dag2 * a2) + d( (a_dag1 * a2 * b) + (a_dag2 * a1 * b_dag) )
    #     c1, c2, d -- constants supplied by user
    #     a_dag1, a1, a_dag2, a2 -- fermionic creation & annihilation operators
    #     b_dag, b -- bosonic creation & annihilation operators

    # Jordan-Wigner Transformation
    #   H = (-1 * c1 / 2 * Z) - (c2 / 2 * Z) + d * ( ((kron(X, X) + kron(Y, Y)) / 4 * (b + b_dag)) + (1j * (kron(X, Y) - kron(Y, X)) / 4 * (b - b_dag)) )
    eye_f = scipy.sparse.csr_matrix([[1, 0], [ 0, 1]])
    eye_b = circuit.ops.eye
    eye_tensor = kron(eye_f, eye_b)

    term1 = kron((-1 * c1 / 2 * Z), eye_tensor)
    term2 = kron((c2 / 2 * Z), eye_tensor)

    xx = kron(X, X)
    yy = kron(Y, Y)
    xx_yy = xx + yy
    term3 = kron((xx_yy / 4), (b + b_dag))

    xy = kron(X, Y)
    yx = kron(Y, X)
    xy_yx = xy - yx
    term4 = kron((1j * (xy_yx / 4)), (b - b_dag))

    tmp1 = term1 - term2
    tmp2 = d * (term3 + term4)

    h = tmp1 + tmp2
    return h


## Loop while reducing time t

In [9]:
# Parameter constants
c1 = 1
c2 = 1
d = 1

results = []

for i in range (100, 0, -10):
    # i = 10
    t = i / 1000

    d1 = (-1j * d * t) / 2
    d2 = (d * t) / 2

    psi = qiskit.quantum_info.random_statevector(2 ** total_qubits)
    psi_matrix = qiskit.quantum_info.DensityMatrix(psi).data

    circuit, state, result = simulate_jaynes_cummings(c1, c2, d1, d2, t, psi=psi)
    #circuit_unitary = result.get_unitary(circuit)  # Need to simulate with 'unitary_simulator' backend to get circuit unitary matrix
    state_matrix = qiskit.quantum_info.DensityMatrix(state).data

    h = calculate_hamiltonian(c1, c2, d, circuit)
    h_psi = h * psi

    # state_psi = (-1j / t) * (numpy.eye(state_matrix.shape[0], state_matrix.shape[1]) - state_matrix) * psi_matrix
    state_psi = (-1j / t) * (state.data - psi.data)
    # state_psi = state.data - psi.data

    result = h_psi + state_psi
    result = numpy.linalg.norm(result)  # TODO -- which norm?
    results.append(result)
    print("****************")
    print(f"Result at t={t}")
    print(result)

/home/richard/anaconda3/envs/C2QAQiskit/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/richard/anaconda3/envs/C2QAQiskit/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


****************
Result at t=0.1
19.26369551338034


/home/richard/anaconda3/envs/C2QAQiskit/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/richard/anaconda3/envs/C2QAQiskit/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


****************
Result at t=0.09
20.80844344237825


/home/richard/anaconda3/envs/C2QAQiskit/lib/python3.9/site-packages/qiskit/extensions/quantum_initializer/squ.py:141: RuntimeWarning: invalid value encountered in arccos
  beta = 2 * np.arccos(np.abs(u00))


****************
Result at t=0.08
14.944221768621128
****************
Result at t=0.07
11.6675212972128
****************
Result at t=0.06
21.17938514753488
****************
Result at t=0.05
39.161949823302926
****************
Result at t=0.04
31.8452072250585
****************
Result at t=0.03
43.6980775196679
****************
Result at t=0.02
96.0134094877775
****************
Result at t=0.01
56.449369659319764
